In [1]:
import sys
sys.path.append('../')
import math
import os
import json
import pickle
import pandas as pd
import numpy as np
from datetime import datetime
from pathlib import Path
import config
import nab_utils

/app/nab
ROOT_PATH = /app


In [2]:
LABELS_FILE=Path(config.NAB_ASSETS) / "labels" / "combined_labels.json"
with open(LABELS_FILE) as json_file:
    labels=json.load(json_file)

def load_raw_datasets(directory, input_filenames):
    datasets={}
    for file in input_filenames:
        datasets[file]=pd.read_csv(directory / file)
    return datasets

def process_datasets(datasets):
    for file in input_filenames:
        datasets[file]['timestamp']=pd.to_datetime(datasets[file]['timestamp']) #set timestamps to datetime objects
        datasets[file]['index']=datasets[file]['timestamp']
        datasets[file]=datasets[file].set_index("index") #set index to datetime column
        datasets[file]["anomaly_score"]=0 # creates new column for labeled anomalies and fills with 0

        for item in labels[file]: #for each entry in the labels for this file
            datasets[file].at[item,"anomaly_score"]=1 #set label to true

input_filenames=nab_utils.get_input_filenames(rootdir = config.NAB_ASSETS / "raw_datasets")
datasets=load_raw_datasets(directory = config.NAB_ASSETS / "raw_datasets" , input_filenames=input_filenames)
process_datasets(datasets)

In [3]:
for file in input_filenames:
    folder = file.split("/")[0]
    filename = "labeler_"+file.split("/")[1]
    save_path=config.NAB_ASSETS / "predictions" / "labeler" / folder 
    Path(save_path).mkdir(parents=True, exist_ok=True) #make folder if doesn't yet exist
    datasets[file].to_csv(Path(save_path / filename).with_suffix('.csv'),index=False)